# Group DARK Project 

In [120]:
import pandas as pd 
from bs4 import BeautifulSoup
import bs4
import tensorflow as tf
import glob
import re
import xml.etree.ElementTree as etree
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB


In [121]:
soup = BeautifulSoup()
path_list = glob.glob('data/*.xml')

data_dicts = []
for f in path_list:
    
     
    soup = BeautifulSoup(open(f), 'html.parser') # I had to use html in order to extract the CData from the XML

    #find the text of the note, which is stored in the CData 
    text = soup.find(text=lambda text: isinstance(text, bs4.CData)).string.strip()

    
    #use regular expression to clean the notes 
    text = re.sub(r'\n+', ' ', text) #remove new paragraph 
    text = re.sub(r'\s\s+\s*', '', text) #remove multiple spaces
    text = re.sub(r'\_+', '', text) #remove underline
    
    #gather smoker status
    try:
        smoker = soup.find('smoker').get('status') #find the smoker status
    except AttributeError:
        smoker = 'unknown'
   
    #find the status of coronary artery disease, 'unknown' will be stored if not known
    try:
        CAD = soup.find('cad').get('indicator')
        
    except AttributeError: 
        
        CAD = 'unknown'
    
    data_dicts.append({'Note': text, 'Smoker': smoker, 'CAD': CAD, 'File name': f})
    
    
    

df = pd.DataFrame(data_dicts)
df = df.drop_duplicates() #in case some files get passed mulitple times, we'll delete any duplicates
        


In [122]:
df.head()

,Note,Smoker,CAD,File name
0,Record date: 2093-04-28BMC EMERGENCY DEPT VISI...,past,unknown,data/379-03.xml
1,Record date: 2093-01-13 Team X Intern Admissio...,unknown,test,data/279-03.xml
2,"Record date: 2088-05-21 Patient Name: CURTIS, ...",unknown,unknown,data/119-01.xml
3,Record date: 2062-03-27 Hematology Clinic Prog...,never,unknown,data/304-03.xml
4,Record date: 2135-12-15CARDIOLOGYPERDUE MEDICA...,past,test,data/204-03.xml


In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1304 entries, 0 to 1303
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Note       1304 non-null   object
 1   Smoker     1304 non-null   object
 2   CAD        1304 non-null   object
 3   File name  1304 non-null   object
dtypes: object(4)
memory usage: 50.9+ KB


In [124]:
df.loc[20]['Note'] #display an example note from the data

"Record date: 2148-07-28 Internal Medicine Intern Admit Note Pt: Samuel Tuttle MR: 2793372 Cc: EKG changesHPI: 66 y.o. male with pmhx of DM, incr.cholestoral, and autonomic insufficeny admitted for EKG changes. The patient has significant risk factors for CAD of DM, incr. Cholesterol, and h/o htn.He had a adenosine stress test done in 11/47 which was negative for ischemia (lasting 5min and PDP of 13746). His last echo was done in 4/48 showed an EF of 59% presented to the health clinic with a 3 day hx of worsening L shoulder pain and episodes of diaphoresis. The shoulder pain is chronic and thought to be due to oa, and dm chor.He came to the clinic today for a BP check ( he is often hypotensive and orthostatic) and was noted to have TWI inferiorly and laterally on routine EKG.He denied any chest pain, nausea, vomiting when he went to the e.r.In the e.r., his first set of cardiac enzymes was negative, he was also started on heparin IV, and given IV lopressor, and an aspirin. Of note, the

In [125]:
df['Smoker'].value_counts()

unknown    635
never      304
past       262
current     91
ever        12
Name: Smoker, dtype: int64

In [126]:
# assuming, 'ever' was a typo of 'never'
df['Smoker'] = df['Smoker'].replace({'ever': 'never'})

In [127]:
df['Smoker'].value_counts() #verify the change took place

unknown    635
never      316
past       262
current     91
Name: Smoker, dtype: int64

In [128]:
df['CAD'].value_counts()

unknown    734
mention    337
test        91
event       83
symptom     59
Name: CAD, dtype: int64

In [129]:
df.to_csv('data.csv', index=False)

In [130]:
df = pd.read_csv('data.csv')
df.head()

,Note,Smoker,CAD,File name
0,Record date: 2093-04-28BMC EMERGENCY DEPT VISI...,past,unknown,data/379-03.xml
1,Record date: 2093-01-13 Team X Intern Admissio...,unknown,test,data/279-03.xml
2,"Record date: 2088-05-21 Patient Name: CURTIS, ...",unknown,unknown,data/119-01.xml
3,Record date: 2062-03-27 Hematology Clinic Prog...,never,unknown,data/304-03.xml
4,Record date: 2135-12-15CARDIOLOGYPERDUE MEDICA...,past,test,data/204-03.xml


In [131]:
df['Smoker'] = df['Smoker'].replace({'unknown': 'unknown', 'never': 'no', 'past': 'yes', 'current': 'yes'})


df.head()

,Note,Smoker,CAD,File name
0,Record date: 2093-04-28BMC EMERGENCY DEPT VISI...,yes,unknown,data/379-03.xml
1,Record date: 2093-01-13 Team X Intern Admissio...,unknown,test,data/279-03.xml
2,"Record date: 2088-05-21 Patient Name: CURTIS, ...",unknown,unknown,data/119-01.xml
3,Record date: 2062-03-27 Hematology Clinic Prog...,no,unknown,data/304-03.xml
4,Record date: 2135-12-15CARDIOLOGYPERDUE MEDICA...,yes,test,data/204-03.xml


In [132]:

#for CAD, we will assume 'unknown' to be someone for which there's no reason to assume coronary artery disease
def binary(row):
    if row == 'unknown':
        val = 'unknown'
    else:
        val = 'yes'
    return val

df['Smoker'] = df['Smoker'].replace(smoker_replacements)

df['CAD'] = df['CAD'].apply(binary)

df.head()

,Note,Smoker,CAD,File name
0,Record date: 2093-04-28BMC EMERGENCY DEPT VISI...,yes,unknown,data/379-03.xml
1,Record date: 2093-01-13 Team X Intern Admissio...,unknown,yes,data/279-03.xml
2,"Record date: 2088-05-21 Patient Name: CURTIS, ...",unknown,unknown,data/119-01.xml
3,Record date: 2062-03-27 Hematology Clinic Prog...,no,unknown,data/304-03.xml
4,Record date: 2135-12-15CARDIOLOGYPERDUE MEDICA...,yes,yes,data/204-03.xml


In [133]:
# export df to csv to make re-running easier
df.to_csv('data.csv', index=False)

In [134]:
#read databack in
df = pd.read_csv('data.csv')

df.head()

,Note,Smoker,CAD,File name
0,Record date: 2093-04-28BMC EMERGENCY DEPT VISI...,yes,unknown,data/379-03.xml
1,Record date: 2093-01-13 Team X Intern Admissio...,unknown,yes,data/279-03.xml
2,"Record date: 2088-05-21 Patient Name: CURTIS, ...",unknown,unknown,data/119-01.xml
3,Record date: 2062-03-27 Hematology Clinic Prog...,no,unknown,data/304-03.xml
4,Record date: 2135-12-15CARDIOLOGYPERDUE MEDICA...,yes,yes,data/204-03.xml


In [135]:
def dummies():
    df = pd.read_csv('data.csv', usecols=['Smoker', 'CAD'])
    new_df = pd.get_dummies(df)
    df = pd.read_csv('data.csv', usecols=['Note'])
    new_df = pd.concat([df, new_df], axis=1)
    return new_df 

In [136]:
new_df = dummies()
new_df.head(10)

,Note,Smoker_no,Smoker_unknown,Smoker_yes,CAD_unknown,CAD_yes
0,Record date: 2093-04-28BMC EMERGENCY DEPT VISI...,0,0,1,1,0
1,Record date: 2093-01-13 Team X Intern Admissio...,0,1,0,0,1
2,"Record date: 2088-05-21 Patient Name: CURTIS, ...",0,1,0,1,0
3,Record date: 2062-03-27 Hematology Clinic Prog...,1,0,0,1,0
4,Record date: 2135-12-15CARDIOLOGYPERDUE MEDICA...,0,0,1,0,1
5,Record date: 2084-01-07Fellow/NP Xiang Reason ...,1,0,0,0,1
6,Record date: 2081-09-06 09/06/81 Follow up aft...,1,0,0,1,0
7,Record date: 2082-10-30 COLIN RAMEY CHH Unit N...,1,0,0,1,0
8,Record date: 2079-11-09 Cocke County Baptist H...,1,0,0,1,0
9,Record date: 2062-07-21 Halcyon House Intern A...,0,1,0,1,0


In [137]:
new_df.to_csv('formatted data.csv', index=False)

In [138]:
new_df = pd.read_csv('formatted data.csv')
new_df.head()

,Note,Smoker_no,Smoker_unknown,Smoker_yes,CAD_unknown,CAD_yes
0,Record date: 2093-04-28BMC EMERGENCY DEPT VISI...,0,0,1,1,0
1,Record date: 2093-01-13 Team X Intern Admissio...,0,1,0,0,1
2,"Record date: 2088-05-21 Patient Name: CURTIS, ...",0,1,0,1,0
3,Record date: 2062-03-27 Hematology Clinic Prog...,1,0,0,1,0
4,Record date: 2135-12-15CARDIOLOGYPERDUE MEDICA...,0,0,1,0,1


In [148]:
from keras.preprocessing.text import text_to_word_sequence
import pandas as pd


def prepareTextFeatures():
    df = pd.read_csv("formatted data.csv",usecols =['Note'])
    texts = df.values
    processeddocs = []
    for idx, sentence in enumerate(texts):
        processedSentence = text_to_word_sequence(sentence[0])
        corpus = ''
        for word in processedSentence:
            corpus = corpus + ' ' + word
        processeddocs.append(corpus)
        
    print(processeddocs[0])
    return processeddocs
corpus = prepareTextFeatures()

 record date 2093 04 28bmc emergency dept visitnancy jasmine nikkolas693 31 18 1visit date 04 28 93 the patient was seen interviewed and examined by myself as well asdr judd whose note i have reviewed and whose findings i haveconfirmed history of presenting complaint this patient is a 68 year oldmale who presents with slurred speech the patient has a historyof a prior cerebrovascular accident with residual right sideddeficit one day prior to arrival the patient was found to haveincreasing slurring of speech and he was found to increasingweakness over his baseline on the right side the patient arriveswithout acute complaints except for difficulty speaking he deniesany headache he also does complain of some throat pain review of systems review of systems is as per the written note the patient denies fevers chills chest pain shortness of breathor dysphagia past medical history the patient's past medical history issignificant for hypertension diabetes mellitus cerebrovascularaccident and o

## RNN

Try an RNN to classify all the labels

In [201]:
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Bidirectional, Flatten, Input, Embedding
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import roc_curve, auc
from keras_tqdm import TQDMNotebookCallback
LABELS = list(new_df.columns)
LABELS = LABELS[1:]


def create_model(voc):
    sequence_input = Input(shape=(100,), dtype='int32')
    embedded_sequences = Embedding(voc, 64, input_length=100)(sequence_input)
    rnn_layer = Bidirectional(LSTM(64, return_sequences=True, dropout=0.3,recurrent_dropout=0),
                              merge_mode='concat')(embedded_sequences)
    l_flat = Flatten()(rnn_layer)
    l_dense = Dense(150, activation='relu')(l_flat)
    l_dense = Dense(120, activation='relu')(l_dense)
    l_dense = Dense(100, activation='relu')(l_dense)
    l_dense = Dense(50, activation='relu')(l_dense)
    l_dense = Dense(50, activation='relu')(l_dense)
    preds = Dense(len(LABELS), activation='sigmoid')(l_dense)
    model = Model(inputs=sequence_input, outputs=preds)
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model
df = pd.read_csv("formatted data.csv")
train, test = train_test_split(df, random_state=42, test_size=0.3, shuffle=True)


X_train = train["Note"].values
X_test = test["Note"].values
Y_train = train[LABELS].values
Y_test = test[LABELS].values




tokenizer = Tokenizer(num_words=10)
tokenizer.fit_on_texts(df["Note"].values)
voc_size = len(tokenizer.word_index)+1


X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train, maxlen=100, padding='post')
voc_size = len(tokenizer.word_index)+1


X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=100, padding='post')

model = create_model(voc_size)

model.fit(x=X_train, y=Y_train, batch_size=128,validation_split=0.3, epochs=10,
          verbose=0,callbacks=[TQDMNotebookCallback(leave_inner=True)])


total=0
prediction = model.predict(X_test)
for idx,category in enumerate(LABELS):
    fpr, tpr, _ = roc_curve(Y_test[:, idx], prediction[:, idx])
    roc_auc = auc(fpr, tpr)
    total+=roc_auc
    print('{}: \n Test auc is {}'.format(category,roc_auc))

print("Macro Average AUC:" + str(total/len(LABELS)))

Model: "model_51"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_53 (InputLayer)        (None, 100)               0         
_________________________________________________________________
embedding_53 (Embedding)     (None, 100, 64)           2887104   
_________________________________________________________________
bidirectional_45 (Bidirectio (None, 100, 128)          66048     
_________________________________________________________________
flatten_51 (Flatten)         (None, 12800)             0         
_________________________________________________________________
dense_140 (Dense)            (None, 150)               1920150   
_________________________________________________________________
dense_141 (Dense)            (None, 120)               18120     
_________________________________________________________________
dense_142 (Dense)            (None, 100)               121

/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "




Smoker_no: 
 Test auc is 0.6959758334232387
Smoker_unknown: 
 Test auc is 0.8240936221231788
Smoker_yes: 
 Test auc is 0.7159346846846847
CAD_unknown: 
 Test auc is 0.5707273064982621
CAD_yes: 
 Test auc is 0.5690025738318253
Macro Average AUC:0.6751468041122379


In [172]:
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Input, Embedding

def create_model(voc):
    sequence_input = Input(shape=(100,), dtype='int32')
    embedded_sequences = Embedding(voc, 64, input_length=100)(sequence_input)
    l_cov1= Conv1D(128, 3, activation='relu')(embedded_sequences)
    l_pool1 = MaxPooling1D(5)(l_cov1)
    l_cov2 = Conv1D(128, 3, activation='relu')(l_pool1)
    l_pool2 = MaxPooling1D(17)(l_cov2)  # global max pooling
    l_flat = Flatten()(l_pool2)
    l_dense = Dense(128, activation='relu')(l_flat)
    preds = Dense(5, activation='sigmoid')(l_dense)
    model = Model(inputs=sequence_input, outputs=preds)
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model


model = create_model(voc_size)

model.fit(x=X_train, y=Y_train,batch_size=158,validation_split=0.2, epochs=10,verbose=0,callbacks=[TQDMNotebookCallback(leave_inner=True)])


total=0
prediction = model.predict(X_test)
for idx,category in enumerate(LABELS):
    fpr, tpr, _ = roc_curve(Y_test[:, idx], prediction[:, idx])
    roc_auc = auc(fpr, tpr)
    total+=roc_auc
    print('{} \n Test auc is {}'.format(category,roc_auc))

print("Macro Average AUC:" + str(total/len(LABELS)))

Model: "model_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_27 (InputLayer)        (None, 100)               0         
_________________________________________________________________
embedding_27 (Embedding)     (None, 100, 64)           2887104   
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 98, 128)           24704     
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 19, 128)           0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 17, 128)           49280     
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 1, 128)            0         
_________________________________________________________________
flatten_25 (Flatten)         (None, 128)               0  

/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "




Smoker_no 
 Test auc is 0.6125930521091811
Smoker_unknown 
 Test auc is 0.757509881422925
Smoker_yes 
 Test auc is 0.672740344092419
CAD_unknown 
 Test auc is 0.5447321669949406
CAD_yes 
 Test auc is 0.5430811416212876
Macro Average AUC:0.6261313172481506


In [72]:
# since this project will require training two neural networks, and we have to drop some values for the smoker model, 
#we will split them into two seperate datasets. 

df_smoker = df[['Note', 'binary smoker']].dropna()

df_smoker.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 669 entries, 0 to 1303
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Note           669 non-null    object 
 1   binary smoker  669 non-null    float64
dtypes: float64(1), object(1)
memory usage: 15.7+ KB


In [73]:
df_CAD = df[['Note', 'binary CAD']]
df_CAD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1304 entries, 0 to 1303
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Note        1304 non-null   object
 1   binary CAD  1304 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 20.5+ KB


In [39]:
#import packages for the tensorflow model
# !pip install -q tensorflow-hub
# !pip install -q tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [74]:
X_C = df_CAD['Note']
y_C = df_CAD['binary CAD']

X_S = df_smoker['Note']
y_S = df_smoker['binary smoker']

In [75]:
#split both of the datasets into testing and training
X_train_C, X_test_C, y_train_C, y_test_C = train_test_split(X_C, y_C, random_state=42, test_size=.2)

X_train_S, X_test_S, y_train_S, y_test_S = train_test_split(X_S, y_S, random_state=42, test_size=.2)

In [81]:
#model architecture. Largely taken from TensorFlow documentation, further hyperparameter tuning will be needed

embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
#first, fit the model to the CAD data

model.fit(X_train_C, y_train_C,
          batch_size=50,
          epochs=30,
          verbose=1,
          validation_data=(X_test_C, y_test_C))

#get classification report to further analyze the model performance
pred = model.predict(X_test_C)
preds = np.where(pred > .5, 1, 0)

print(classification_report(y_test_C, preds, digits=2))
print(confusion_matrix(y_test_C, preds))

Train on 1043 samples, validate on 261 samples
Epoch 1/30
1043/1043 [==============================] - 1s 931us/sample - loss: 0.8464 - accuracy: 0.5561 - val_loss: 0.6868 - val_accuracy: 0.5977
Epoch 2/30
1043/1043 [==============================] - 0s 428us/sample - loss: 0.7177 - accuracy: 0.5657 - val_loss: 0.6742 - val_accuracy: 0.5977
Epoch 3/30
1043/1043 [==============================] - 0s 405us/sample - loss: 0.6921 - accuracy: 0.5724 - val_loss: 0.6689 - val_accuracy: 0.5977
Epoch 4/30
1043/1043 [==============================] - 0s 379us/sample - loss: 0.6729 - accuracy: 0.5781 - val_loss: 0.6636 - val_accuracy: 0.5939
Epoch 5/30
1043/1043 [==============================] - 0s 454us/sample - loss: 0.6571 - accuracy: 0.5877 - val_loss: 0.6564 - val_accuracy: 0.5977
Epoch 6/30
1043/1043 [==============================] - 0s 451us/sample - loss: 0.6411 - accuracy: 0.6002 - val_loss: 0.6503 - val_accuracy: 0.6169
Epoch 7/30
1043/1043 [==============================] - 1s 525us/

In [82]:
#Now fit the model to the smoker data

embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

model_S = tf.keras.Sequential()
model_S.add(hub_layer)
model_S.add(tf.keras.layers.Dense(4, activation='relu'))
model_S.add(tf.keras.layers.Dense(1))
model_S.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_S.fit(X_train_S, y_train_S,
          batch_size=50,
          epochs=70,
          verbose=1,
          validation_data=(X_test_S, y_test_S))

#get classification report to further analyze the model performance
pred = model_S.predict(X_test_S)
preds = np.where(pred > .5, 1, 0)

print(classification_report(y_test_S, preds, digits=2))
print(confusion_matrix(y_test_S, preds))

Train on 535 samples, validate on 134 samples
Epoch 1/70
535/535 [==============================] - 1s 2ms/sample - loss: 2.6770 - accuracy: 0.4785 - val_loss: 1.8040 - val_accuracy: 0.4925
Epoch 2/70
535/535 [==============================] - 0s 558us/sample - loss: 1.5929 - accuracy: 0.4841 - val_loss: 1.1912 - val_accuracy: 0.5448
Epoch 3/70
535/535 [==============================] - 0s 669us/sample - loss: 1.0878 - accuracy: 0.5364 - val_loss: 0.9952 - val_accuracy: 0.5746
Epoch 4/70
535/535 [==============================] - 0s 630us/sample - loss: 0.9562 - accuracy: 0.5477 - val_loss: 0.9213 - val_accuracy: 0.5597
Epoch 5/70
535/535 [==============================] - 0s 454us/sample - loss: 0.8733 - accuracy: 0.5458 - val_loss: 0.8423 - val_accuracy: 0.5597
Epoch 6/70
535/535 [==============================] - 0s 460us/sample - loss: 0.8000 - accuracy: 0.5589 - val_loss: 0.7998 - val_accuracy: 0.5373
Epoch 7/70
535/535 [==============================] - 0s 507us/sample - loss: 0.

In [67]:
# Try Naive Bayes MultinomialNB classifier with TfidfVecorizer 
stop_words = set(stopwords.words('english')) #define the stopwords

#vectorizer = TfidfVectorizer(max_df= 2.0, min_df=2, stop_words=stop_words)

vectorizer = CountVectorizer(analyzer='word', stop_words=stop_words)

X_C = vectorizer.fit_transform(df_CAD['Note'])
#y_C already defined

#re-split the data
X_train_C, X_test_C, y_train_C, y_test_C = train_test_split(X_C, y_C, random_state=42, test_size=.2)


clf = MultinomialNB(alpha=0)
clf.fit(X_train_C, y_train_C)
y_pred = clf.predict(X_test_C)

print(f'Accuracy: {accuracy_score(y_test_C, y_pred)}')
print(f'\nClassification report:\n{classification_report(y_test_C, y_pred)}')
print(f'\nConfusion matrix: \n{confusion_matrix(y_test_C, y_pred)}')


Accuracy: 0.7854406130268199

Classification report:
              precision    recall  f1-score   support

           0       0.83      0.79      0.81       150
           1       0.74      0.77      0.75       111

    accuracy                           0.79       261
   macro avg       0.78      0.78      0.78       261
weighted avg       0.79      0.79      0.79       261


Confusion matrix: 
[[119  31]
 [ 25  86]]


/opt/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


In [68]:
# Try the MultinomialNB on the smoker data 

stop_words = set(stopwords.words('english')) #define the stopwords

vectorizer = TfidfVectorizer(max_df= 2.0, min_df=2, stop_words=stop_words)

#vectorizer = CountVectorizer(stop_words=stop_words)

X_S = vectorizer.fit_transform(df_smoker['Note'])
#y_S already defined

#re-split the data
X_train_S, X_test_S, y_train_S, y_test_S = train_test_split(X_S, y_S, random_state=42, test_size=.2)


clf = MultinomialNB(alpha=0)
clf.fit(X_train_S, y_train_S)
y_pred = clf.predict(X_test_S)

print(f'Accuracy: {accuracy_score(y_test_S, y_pred)}')
print(f'\nClassification report:\n{classification_report(y_test_S, y_pred)}')
print(f'\nConfusion matrix: \n{confusion_matrix(y_test_S, y_pred)}')

Accuracy: 0.6716417910447762

Classification report:
              precision    recall  f1-score   support

         0.0       0.65      0.63      0.64        62
         1.0       0.69      0.71      0.70        72

    accuracy                           0.67       134
   macro avg       0.67      0.67      0.67       134
weighted avg       0.67      0.67      0.67       134


Confusion matrix: 
[[39 23]
 [21 51]]


/opt/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
